In [15]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from datetime import datetime
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

In [16]:
import apyori as ap

In [17]:
from tqdm import tqdm

In [18]:
transaction2 = pd.read_csv('clac_nm.csv', engine='python', encoding='utf-8-sig')

In [19]:
transaction3 = transaction2.pivot_table(index=['clnt_id', 'de_dt'], columns='clac_nm0', values='buy_ct', aggfunc='sum').fillna(0).reset_index(['clnt_id', 'de_dt']).drop(['clnt_id', 'de_dt'], axis=1)

In [20]:
nm0_list = []
nm0 = []

for i in tqdm(range(0, len(transaction3))):
  for j in range(0, len(transaction3.columns)):
    if transaction3.loc[i][j] > 0:
      nm0.append(j)
    else:
      continue
  nm0_list.append(nm0)
  nm0 = []

100%|███████████████████████████████████████████████████████████████████████████| 94376/94376 [03:59<00:00, 394.00it/s]


In [21]:
for i in tqdm(range (0, len(nm0_list))):
  for j in range(0, len(nm0_list[i])):
    nm0_list[i][j]=str(nm0_list[i][j])

    if nm0_list[i][j] == '1':
      nm0_list[i][j]=nm0_list[i][j].replace('1', 'Baby')
    elif nm0_list[i][j] == '2':
      nm0_list[i][j]=nm0_list[i][j].replace('2', 'Beauty')    
    elif nm0_list[i][j] == '3':
      nm0_list[i][j]=nm0_list[i][j].replace('3','Digiter, Appliences')
    elif nm0_list[i][j] == '4':
      nm0_list[i][j]=nm0_list[i][j].replace('4', 'Fashion_Accessories') 
    elif nm0_list[i][j] == '5':
      nm0_list[i][j]=nm0_list[i][j].replace('5', 'Fashion_Clothes')    
    elif nm0_list[i][j] == '6':
      nm0_list[i][j]=nm0_list[i][j].replace('6','Foods')
    elif nm0_list[i][j] == '7':
      nm0_list[i][j]=nm0_list[i][j].replace('7', 'Interier')  
    elif nm0_list[i][j] == '8':
      nm0_list[i][j]=nm0_list[i][j].replace('8', 'Living, Health')    
    elif nm0_list[i][j] == '9':
      nm0_list[i][j]=nm0_list[i][j].replace('9', 'Others')
    elif nm0_list[i][j] == '10':
      nm0_list[i][j]=nm0_list[i][j].replace('10', 'Sports, Leisure')   
    elif nm0_list[i][j] == '11':
      nm0_list[i][j]=nm0_list[i][j].replace('11', 'Travel, Culture')

100%|████████████████████████████████████████████████████████████████████████| 94376/94376 [00:00<00:00, 198382.25it/s]


In [34]:
rules = ap.apriori(nm0_list, min_support=0.01, min_confidence=0.1)

In [35]:
table=[]

for rule in list(rules):
    support = rule[1]
    infor = list(rule[2])
    if len(infor) > 1:
        for i in infor:
            row = str(set(i[0])).replace('\'','').replace('{','').replace('}',''),\
            "==>", \
            str(set(i[1])).replace('\'','').replace('{','').replace('}',''),\
            support, i[2], i[3]
            table.append(row)
            
df = pd.DataFrame(data = table, \
             columns = ['base', '->', 'add', 'support', 'confidence', 'lift'], \
             index=range(1,len(table)+1))

In [36]:
df

,base,->,add,support,confidence,lift
1,Baby,==>,Foods,0.015608,0.143260,1.133969
2,Foods,==>,Baby,0.015608,0.123543,1.133969
3,Baby,==>,Interier,0.033038,0.303248,1.437726
4,Interier,==>,Baby,0.033038,0.156636,1.437726
5,Fashion_Clothes,==>,Foods,0.114139,0.140537,1.112413
6,Foods,==>,Fashion_Clothes,0.114139,0.903464,1.112413
7,Fashion_Clothes,==>,Interier,0.170997,0.210544,0.998207
8,Interier,==>,Fashion_Clothes,0.170997,0.810710,0.998207
9,Foods,==>,Interier,0.037690,0.298331,1.414412
10,Interier,==>,Foods,0.037690,0.178690,1.414412
